In [1]:
# General
import os
import numpy as np
import sys
from glob import glob
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#ML
import tensorflow as tf
from tensorflow.keras import mixed_precision, layers
mixed_precision.set_global_policy("mixed_float16")
AUTO = tf.data.experimental.AUTOTUNE

In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
Params = {
    "batch_size": 128
}

In [4]:
def random_cut(x,y):
    tensor = x
    tensor_y = y
    maxVal=128
    dt = tf.random.uniform(shape=[],minval=2, maxval=maxVal, dtype=tf.int32)
    t0 = tf.random.uniform(shape=[],minval=1, maxval=dt, dtype=tf.int32)
    t1 = tf.random.uniform(shape=[],minval=0, maxval=t0, dtype=tf.int32)
    paddings =  [
        [0,0],
        [t1,dt-t1],
        [0,0]
    ]
    tensor = tf.pad(tensor[:,t0:t0+(4096-dt)], paddings=paddings)
    tensor_y = tf.pad(tensor_y[:,t0:t0+4096-dt], paddings=paddings)
    tensor = tf.reshape(tensor,[Params["batch_size"], 4096, 3])
    tensor_y = tf.reshape(tensor_y,[Params["batch_size"], 4096, 3])
        
    
    tensor = tf.cast(tensor, tf.float32)
    tensor_y = tf.cast(tensor_y, tf.float32)
    
    return tensor, tensor_y

In [5]:
def load_dataset(files, window=True):
    dataset = tf.data.TFRecordDataset(files, num_parallel_reads = AUTO)
    
    def _parse_function(example_proto):
        keys_to_feature = {}
        keys_to_feature["TimeSeries"] = tf.io.FixedLenFeature([4096,3], tf.float32)
        keys_to_feature["GroundTruths"] = tf.io.FixedLenFeature([4096,3], tf.float32)
        
        parsed_features = tf.io.parse_single_example(example_proto, keys_to_feature)
        return parsed_features["TimeSeries"], parsed_features["GroundTruths"]
    
    dataset = dataset.map(_parse_function, num_parallel_calls=AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(buffer_size=10000, reshuffle_each_iteration=True)
    dataset = dataset.batch(Params["batch_size"])
    dataset = dataset.map(random_cut, num_parallel_calls=AUTO)
#     dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset

In [6]:
def get_dataset_files(distances):
    train_files = []
    val_files = []
    for distance in distances:
        data_files = glob(f"../input/synthetic-tfrec/train_{distance}*.tfrec")
        train_files.extend(data_files[:-1])
        val_files.extend(data_files[-1:])
    return train_files, val_files

In [7]:
train_files,val_files = get_dataset_files([100,150,200,250,300,350])
train_ds = load_dataset(train_files)
val_ds = load_dataset(val_files)
train_files

['../input/synthetic-tfrec\\train_100_12500_00.tfrec',
 '../input/synthetic-tfrec\\train_100_12500_01.tfrec',
 '../input/synthetic-tfrec\\train_100_12500_02.tfrec',
 '../input/synthetic-tfrec\\train_100_12500_03.tfrec',
 '../input/synthetic-tfrec\\train_100_12500_04.tfrec',
 '../input/synthetic-tfrec\\train_100_12500_05.tfrec',
 '../input/synthetic-tfrec\\train_100_12500_06.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_00.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_01.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_02.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_03.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_04.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_05.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_06.tfrec',
 '../input/synthetic-tfrec\\train_200_12500_00.tfrec',
 '../input/synthetic-tfrec\\train_200_12500_01.tfrec',
 '../input/synthetic-tfrec\\train_200_12500_02.tfrec',
 '../input/synthetic-tfrec\\train_200_12500_03.tfrec',
 '../input

In [8]:
def ownInitializer(shape, dtype=None):
    return tf.constant([
        [[1,0,0,0]],
        [[0,1,0,0]],
        [[0,0,1,0]],
        [[0,0,0,1]]
    ],dtype=dtype)

ownInitializer(2,tf.float32)

<tf.Tensor: shape=(4, 1, 4), dtype=float32, numpy=
array([[[1., 0., 0., 0.]],

       [[0., 1., 0., 0.]],

       [[0., 0., 1., 0.]],

       [[0., 0., 0., 1.]]], dtype=float32)>

In [9]:
model = tf.keras.Sequential([
    tf.keras.Input(shape = [4096,3]),
#         layers.Conv1D(kernel_size=12, filters=4,
#                        kernel_initializer=ownInitializer, input_shape=[4096,3], trainable=True, padding="same"),
    layers.Conv1D(kernel_size=4, filters=12, padding="same"),
#     layers.ZeroPadding1D(padding=[0,3]),
    layers.Reshape([4096,12,1]),
    layers.TimeDistributed(layers.Conv1D(kernel_size=3, filters=32, activation="tanh", padding="same")),
#     layers.TimeDistributed(layers.Conv1D(kernel_size=5, filters=32, activation="tanh", padding="same")),
    
    layers.TimeDistributed(layers.MaxPool1D()),
    layers.TimeDistributed(layers.Conv1D(kernel_size=3, filters=16, activation="tanh", padding="same")),
    layers.TimeDistributed(layers.Flatten()),
    
#     layers.Reshape([4093,32]),
    layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128, return_sequences=True)),
    layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128, return_sequences=True)),
    layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128, return_sequences=True)),
    
#     layers.Bidirectional(layers.LSTM(100, return_sequences=True)),
#     layers.Bidirectional(layers.LSTM(100, return_sequences=True)),
#     layers.Bidirectional(layers.LSTM(100, return_sequences=True)),
    
    
#     layers.Bidirectional(layers.LSTM(400)),
    
    
    layers.TimeDistributed(layers.Dense(3, dtype=tf.float32)),
])
opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)

model.compile(
    optimizer=opt,
    metrics=["mean_squared_error", "cosine_similarity"],
    loss="mean_squared_error"
)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 4096, 12)          156       
_________________________________________________________________
reshape (Reshape)            (None, 4096, 12, 1)       0         
_________________________________________________________________
time_distributed (TimeDistri (None, 4096, 12, 32)      192       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4096, 12, 32)      5152      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4096, 6, 32)       0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 4096, 6, 16)       2576      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 4096, 96)          

In [10]:
# model=tf.keras.models.load_model("model2.h5")

In [11]:
train_steps = len(train_files) * 12500 // Params["batch_size"]
val_steps = len(val_files) * 12500 // Params["batch_size"]

In [12]:
from tensorflow.keras.callbacks import *
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.2,
    patience=5, min_lr = 0.000001,
    verbose=1
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=10
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
#     f"{MDL_PATH}/model_{Params['version']:03}.h5",
    "./model4.h5",
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weight_only=False,
    mode="auto",
    save_freq="epoch"
)


callbacks=[reduce_lr, early_stop, model_checkpoint]

In [ ]:
model.fit(train_ds,validation_data=val_ds, validation_steps=val_steps, steps_per_epoch=train_steps,
          epochs=60, callbacks=callbacks)

Epoch 1/60
4783/7421 [==================>...........] - ETA: 36:26 - loss: 0.0108 - mean_squared_error: 0.0108 - cosine_similarity: 0.0423

In [ ]:

predictions = model.predict(train_ds, steps=5)

In [ ]:
predictions.shape

In [ ]:
import matplotlib.pyplot as plt
for i in range(1000,1050):
    
    plt.plot(predictions[i])
    plt.show()

In [ ]:
for i in train_ds:
    plt.plot(i[1][7])
    break
    

In [ ]:
model.save("model2.h5")